In [2]:
import torch
import numpy as np

In [41]:
english_file = '../data/train.en'
sanskrit_file = '../data/train.sn'

START_TOKEN = ''
PADDING_TOKEN = ''
END_TOKEN = ''

sanskrit_vocabulary = [
    START_TOKEN, ' ', '!', '"', "'", '(', ')', ',', '-', '.', '?', ':', ';',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',

    # Independent vowels
    'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ॠ', 'ऌ', 'ॡ', 'ए', 'ऐ', 'ओ', 'औ',
    
    # Consonants
    'क', 'ख', 'ग', 'घ', 'ङ',
    'च', 'छ', 'ज', 'झ', 'ञ',
    'ट', 'ठ', 'ड', 'ढ', 'ण',
    'त', 'थ', 'द', 'ध', 'न',
    'प', 'फ', 'ब', 'भ', 'म',
    'य', 'र', 'ल', 'व',
    'श', 'ष', 'स', 'ह',

    # Vowel signs
    'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॄ', 'े', 'ै', 'ो', 'ौ',

    # Other signs
    'ं', 'ः', 'ँ', '्',  # virama, anusvara, visarga, chandrabindu
    '।', '॥',  # danda marks

    # Special tokens
    PADDING_TOKEN, END_TOKEN
]

english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@', 
                        'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 
                        'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 
                        'Y', 'Z',
                            # Diacritic letters used in IAST transliteration
    'ā', 'ī', 'ū', 'ṛ', 'ṝ', 'ḷ', 'ḹ',
    'ṅ', 'ñ', 'ṭ', 'ḍ', 'ṇ',
    'ś', 'ṣ', 'ḻ',

    # Capital versions (if any)
    'Ā', 'Ī', 'Ū', 'Ṛ', 'Ṝ', 'Ḷ', 'Ḹ',
    'Ṅ', 'Ñ', 'Ṭ', 'Ḍ', 'Ṇ',
    'Ś', 'Ṣ', 'Ḻ',
                        '_',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 
                        'y', 'z', 
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

In [42]:
text = 'Şītā'
list(text)

['Ş', 'ī', 't', 'ā']

In [43]:
text = 'ममैवाभिमुखं'
list(text)

['म', 'म', 'ै', 'व', 'ा', 'भ', 'ि', 'म', 'ु', 'ख', 'ं']

In [44]:

'म'+ 'ै'

'मै'

In [45]:
english_file

'../data/train.en'

In [46]:

index_to_sanskrit = {k:v for k,v in enumerate(sanskrit_vocabulary)}
sanskrit_to_index = {v:k for k,v in enumerate(sanskrit_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}
     

In [47]:
with open(english_file, 'r') as file:
    english_sentences = file.readlines()
with open(sanskrit_file, 'r') as file:
    sanskrit_sentences = file.readlines()

# Limit Number of sentences
TOTAL_SENTENCES = 100000
english_sentences = english_sentences[:TOTAL_SENTENCES]
sanskrit_sentences = sanskrit_sentences[:TOTAL_SENTENCES]
english_sentences = [sentence.rstrip('\n') for sentence in english_sentences]
sanskrit_sentences = [sentence.rstrip('\n') for sentence in sanskrit_sentences]

In [48]:

english_sentences[:10]

['When Şītā, having a husband although seeming as if she had none, was putting on the ascetic guise, the people got into a wrath and exclaimed, “O Dasaratha, fie on you!"',
 'Aggrieved at the uproar that arose there in consequence, the lord of earth banished from his heart all regard for life, virtue, and fame. And sighing hot, that descendant of Ikşvāku spoke to that wife of his, saying, O Kaikeyi, Sītā deserves not to go in a Kuća dress.',
 'Tender, and youthful, and worthy of happiness, she is by no means capable of living in the forest. My spiritual guide has spoken the truth.',
 'Whom has this one injured that, being the daughter of the foremost of kings, she like a female ascetic, wearing a meagre garb in the presence of all, will (repair to the woods and) remain there like a beggar destitute of everything?',
 "Let Janaka's daughter leave off her ascetic guise. This is not the promise that I had made to you before. Let the princess go to the forest in comfort, furnished with all 

In [49]:
sanskrit_sentences[:10]

['तस्यां चीरं वसानायां नाथवत्यामनाथवत्। प्रचुक्रोश जनः सर्वो धिक् त्वां दशरथं त्विति ॥',
 'तेन तत्र प्रणादेन दुःखितः स महीपतिः। चिच्छेद जीविते श्रद्धां धर्मे यशसि चात्मनः॥ स निःश्वस्योष्णमैक्ष्वाकस्तां भार्यामिदमब्रवीत्। कैकेयि कुशचीरेण न सीता गन्तुमर्हति॥',
 'सुकुमारी च बाला च सततं च सुखोचिता। नेयं वनस्य योग्येति सत्यमाह गुरुर्मम ॥',
 'इयं हि कस्यापि करोति किंचित् तपस्विनी राजवरस्य पुत्री। या चीरमासाद्य वनस्य मध्ये जाता विसंज्ञा श्रमणीव काचित्॥',
 'चीराण्यपास्याज्जनकस्य कन्या नेयं प्रतिज्ञा मम दत्तपूर्वा। यथासुखं गच्छतु राजपुत्री वनं समग्रा सह सर्वरत्नैः॥',
 'अजीवनाहेण मया नृशंसा कृता प्रतिज्ञा नियमेन तावत्। त्वया हि बाल्यात् प्रतिपन्नमेतत् तन्मा दहेद् वेणुमिवात्मपुष्पम्॥',
 'रामेण यदि ते पापे किंचित्कृतमशोभनम्। अपकारः क इह ते वैदेह्या दर्शितोऽधमे॥',
 'मृगीवोत्फुल्लनयना मृदुशीला मनस्विनी। अपकारं कमिव ते करोति जनकात्मजा॥',
 'ननु पर्याप्तमेवं ते पापे रामविवासनम्। किमेभिः कृपणैर्भूयः पातकैरपि ते कृतैः॥',
 'प्रतिज्ञातं मया तावत् त्वयोक्तं देवि शृण्वता। रामं यदभिषेकाय त्वमिहागतमब्रवीः॥']

In [50]:

max(len(x) for x in sanskrit_sentences), max(len(x) for x in english_sentences),

(5294, 12194)

In [51]:

PERCENTILE = 97
print( f"{PERCENTILE}th percentile length sanskrit: {np.percentile([len(x) for x in sanskrit_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )

97th percentile length sanskrit: 207.78999999999996
97th percentile length English: 381.78999999999996


In [52]:
max_sequence_length = 400

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(sanskrit_sentences)):
    sanskrit_sentence, english_sentence = sanskrit_sentences[index], english_sentences[index]
    if is_valid_length(sanskrit_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(sanskrit_sentence, sanskrit_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(sanskrit_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 7108
Number of valid sentences: 5540


In [53]:
sanskrit_sentences = [sanskrit_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [54]:
sanskrit_sentences[:3]

['तस्यां चीरं वसानायां नाथवत्यामनाथवत्। प्रचुक्रोश जनः सर्वो धिक् त्वां दशरथं त्विति ॥',
 'तेन तत्र प्रणादेन दुःखितः स महीपतिः। चिच्छेद जीविते श्रद्धां धर्मे यशसि चात्मनः॥ स निःश्वस्योष्णमैक्ष्वाकस्तां भार्यामिदमब्रवीत्। कैकेयि कुशचीरेण न सीता गन्तुमर्हति॥',
 'सुकुमारी च बाला च सततं च सुखोचिता। नेयं वनस्य योग्येति सत्यमाह गुरुर्मम ॥']

In [55]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, sanskrit_sentences):
        self.english_sentences = english_sentences
        self.sanskrit_sentences = sanskrit_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.sanskrit_sentences[idx]

In [56]:
dataset = TextDataset(english_sentences, sanskrit_sentences)

In [57]:

len(dataset)

5540

In [58]:

dataset[1]

('Aggrieved at the uproar that arose there in consequence, the lord of earth banished from his heart all regard for life, virtue, and fame. And sighing hot, that descendant of Ikşvāku spoke to that wife of his, saying, O Kaikeyi, Sītā deserves not to go in a Kuća dress.',
 'तेन तत्र प्रणादेन दुःखितः स महीपतिः। चिच्छेद जीविते श्रद्धां धर्मे यशसि चात्मनः॥ स निःश्वस्योष्णमैक्ष्वाकस्तां भार्यामिदमब्रवीत्। कैकेयि कुशचीरेण न सीता गन्तुमर्हति॥')

In [59]:

batch_size = 3 
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [60]:

for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 3:
        break

[('When Şītā, having a husband although seeming as if she had none, was putting on the ascetic guise, the people got into a wrath and exclaimed, “O Dasaratha, fie on you!"', 'Aggrieved at the uproar that arose there in consequence, the lord of earth banished from his heart all regard for life, virtue, and fame. And sighing hot, that descendant of Ikşvāku spoke to that wife of his, saying, O Kaikeyi, Sītā deserves not to go in a Kuća dress.', 'Tender, and youthful, and worthy of happiness, she is by no means capable of living in the forest. My spiritual guide has spoken the truth.'), ('तस्यां चीरं वसानायां नाथवत्यामनाथवत्। प्रचुक्रोश जनः सर्वो धिक् त्वां दशरथं त्विति ॥', 'तेन तत्र प्रणादेन दुःखितः स महीपतिः। चिच्छेद जीविते श्रद्धां धर्मे यशसि चात्मनः॥ स निःश्वस्योष्णमैक्ष्वाकस्तां भार्यामिदमब्रवीत्। कैकेयि कुशचीरेण न सीता गन्तुमर्हति॥', 'सुकुमारी च बाला च सततं च सुखोचिता। नेयं वनस्य योग्येति सत्यमाह गुरुर्मम ॥')]
[('Whom has this one injured that, being the daughter of the foremost of k

In [61]:
def tokenize(sentence, language_to_index, start_token=True, end_token=True):
    sentence_word_indicies = [language_to_index[token] for token in list(sentence)]
    if start_token:
        sentence_word_indicies.insert(0, language_to_index[START_TOKEN])
    if end_token:
        sentence_word_indicies.append(language_to_index[END_TOKEN])
    for _ in range(len(sentence_word_indicies), max_sequence_length):
        sentence_word_indicies.append(language_to_index[PADDING_TOKEN])
    return torch.tensor(sentence_word_indicies)

In [62]:


batch

[('O you comparable to the mighty Indra, you should so behove with my mother smitten with my separation, that exercised by grief in consequence of my residence in the forest, she may not, renouncing life, repair to the mansions of Yama.',
  'Hearing those bold words of the ascetic, the son of that foremost of men, Rāghava firm in his vows, with clasped hands answered.',
  'And in the interests of Brāhmaṇas, kine, and celestials, I am ready to act as desired by you of immeasurable energy.'),
 ('पुत्रशोकं यथा नछेत् त्वया पूज्येन पूजिता। मां हि संचिन्तयन्ती सा त्वयि जीवेत् तपस्विनी॥ इमां महेन्द्रोपम जातगर्धिनी तथा विधातुं जननी ममार्हसि। यथा वनस्थे मयि शोककर्शिता न जीवितं न्यस्य यमक्षयं व्रजेत्॥',
  'मुनेर्वचनमक्लीबं श्रुत्वा नरवरात्मजः। राघवः प्राञ्जलिर्भूत्वा प्रत्युवाच दृढव्रतः॥',
  'गोब्राह्मणहितार्थाय देशस्य च हिताय च। तव चैवाप्रमेयस्य वचनं कर्तुमुद्यतः॥')]

In [64]:
# batch[sentence_num]

NameError: name 'sentence_num' is not defined

In [65]:
eng_tokenized, kn_tokenized = [], []
for sentence_num in range(batch_size):
    eng_sentence, kn_sentence = batch[0][sentence_num], batch[1][sentence_num]
    eng_tokenized.append( tokenize(eng_sentence, english_to_index, start_token=False, end_token=False) )
    kn_tokenized.append( tokenize(kn_sentence, sanskrit_to_index, start_token=True, end_token=True) )
eng_tokenized = torch.stack(eng_tokenized)
kn_tokenized = torch.stack(kn_tokenized)

In [ ]:
## For Sanskrit to English translation, you can use the following code:
# eng_tokenized, kn_tokenized = [], []
# for sentence_num in range(batch_size):
#     eng_sentence, kn_sentence = batch[0][sentence_num], batch[1][sentence_num]
#     eng_tokenized.append( tokenize(eng_sentence, english_to_index, start_token=True, end_token=True) )
#     kn_tokenized.append( tokenize(kn_sentence, sanskrit_to_index, start_token=False, end_token=False) )
# eng_tokenized = torch.stack(eng_tokenized)
# kn_tokenized = torch.stack(kn_tokenized)

In [68]:

eng_tokenized

tensor([[ 47,   1, 114,  ..., 121, 121, 121],
        [ 40,  94,  90,  ..., 121, 121, 121],
        [ 33, 103,  93,  ..., 121, 121, 121]])

In [69]:
NEG_INFTY = -1e9

def create_masks(eng_batch, kn_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, kn_sentence_length = len(eng_batch[idx]), len(kn_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      kn_chars_to_padding_mask = np.arange(kn_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, kn_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, kn_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, kn_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    print(f"encoder_self_attention_mask {encoder_self_attention_mask.size()}: {encoder_self_attention_mask[0, :10, :10]}")
    print(f"decoder_self_attention_mask {decoder_self_attention_mask.size()}: {decoder_self_attention_mask[0, :10, :10]}")
    print(f"decoder_cross_attention_mask {decoder_cross_attention_mask.size()}: {decoder_cross_attention_mask[0, :10, :10]}")
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [70]:

create_masks(batch[0], batch[1])
     

encoder_self_attention_mask torch.Size([3, 400, 400]): tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
decoder_self_attention_mask torch.Size([3, 400, 400]): tensor([[ 0.0000e+00, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09,
         -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
        [ 0.0000e+00,  0.0000e+00, -1.0000e+09, -1.0000e+09, -1.0000e+09,
         -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00, -1.0000e+09, -1.0000e+09,
         -1.0000e

(tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          ...,
          [-1.0000e+09, -1.0000e+09, -1.0000e+09,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          [-1.0000e+09, -1.0000e+09, -1.0000e+09,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          [-1.0000e+09, -1.0000e+09, -1.0000e+09,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09]],
 
         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -1.0000e+09,
           -1.0000e+09, -1.0000e+09],
          ...,
    

In [72]:
import torch.nn as nn

In [73]:
class SentenceEmbedding(nn.Module):
    "For a given sentence, create an embedding"
    def __init__(self, max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN):
        super().__init__()
        self.vocab_size = len(language_to_index)
        self.max_sequence_length = max_sequence_length
        self.embedding = nn.Embedding(self.vocab_size, d_model)
        self.language_to_index = language_to_index
        self.position_encoder = PositionalEncoding(d_model, max_sequence_length)
        self.dropout = nn.Dropout(p=0.1)
        self.START_TOKEN = START_TOKEN
        self.END_TOKEN = END_TOKEN
        self.PADDING_TOKEN = PADDING_TOKEN
    
    def batch_tokenize(self, batch, start_token=True, end_token=True):

        def tokenize(sentence, start_token=True, end_token=True):
            sentence_word_indicies = [self.language_to_index[token] for token in list(sentence)]
            if start_token:
                sentence_word_indicies.insert(0, self.language_to_index[self.START_TOKEN])
            if end_token:
                sentence_word_indicies.append(self.language_to_index[self.END_TOKEN])
            for _ in range(len(sentence_word_indicies), self.max_sequence_length):
                sentence_word_indicies.append(self.language_to_index[self.PADDING_TOKEN])
            return torch.tensor(sentence_word_indicies)

        tokenized = []
        for sentence_num in range(len(batch)):
           tokenized.append( tokenize(batch[sentence_num], start_token, end_token) )
        tokenized = torch.stack(tokenized)
        return tokenized.to(get_device())
    
    def forward(self, x, end_token=True): # sentence
        x = self.batch_tokenize(x ,end_token)
        x = self.embedding(x)
        pos = self.position_encoder().to(get_device())
        x = self.dropout(x + pos)
        return x
